In [10]:
# Create a ward/precinct to BOE 20 district intersection file

import pandas as pd
import geopandas as gpd
import os
import folium

mdir = os.getcwd()

df = pd.read_csv(rf"{mdir}\\data.csv")

# Split df into different datasets

 # Strip whitespace from column names

df['race_name'] = [xstr.strip() for xstr in df['race_name']]

reg_voters = df[df['race_name'].isin(['Total Registered Voters'])]
ballots_cast = df[df['race_name'].isin(['Ballots Cast'])]
boe = df[df['race_name'].str.contains('Member of the Chicago Board of Education')]
pt_relief = df[df['race_name'].isin(['The Property Tax Relief and Fairness Referendum'])]

reg_voters = reg_voters[["ward","precinct","votes"]]
reg_voters.columns = ["ward","precinct","registered_voters"]
ballots_cast = ballots_cast[["ward","precinct","votes"]]
ballots_cast.columns = ["ward","precinct","ballots_cast"]

general = reg_voters.merge(ballots_cast,on=["ward","precinct"],how="left")

general['ward_precinct'] = general['ward'].apply(lambda x: str(x).zfill(2)) + general['precinct'].apply(lambda x: str(x).zfill(3))

# District and precinct totals for candidates and BOE races
boe_district = boe.groupby(['race_name']).agg({
    'votes':'sum'
    }).reset_index()
boe_candidate_total = boe.groupby(['race_name','candidate_name']).agg({
    'votes':'sum'
    }).reset_index()
boe_precinct = boe.groupby(['race_name','ward','precinct']).agg({
    'votes':'sum'
    }).reset_index()

boe = boe.merge(boe_district,on="race_name",how="left",suffixes=("","_district"))
boe = boe.merge(boe_candidate_total,on=["race_name","candidate_name"],how="left",suffixes=("","_candidate_district_total"))
boe = boe.merge(boe_precinct,on=["race_name","ward","precinct"],how="left",suffixes=("","_precinct"))

# Rename for clarity
rename = {
    'votes':'ballots_candidate_precinct',
    'votes_district':'ballots_district_total',
    'votes_candidate_district_total':'ballots_candidate_district_total',
    'votes_precinct':'ballots_precinct_total'
}

boe = boe.rename(columns=rename)

# pad ward with 1 leading zero and precinct with 2 leading zeros

boe['ward'] = boe['ward'].apply(lambda x: str(x).zfill(2))
boe['precinct'] = boe['precinct'].apply(lambda x: str(x).zfill(3))
boe['ward_precinct'] = boe['ward'] + boe['precinct']

# for race in boe['race_name'].unique():

#     df = boe[boe['race_name'] == race]
#     df = df.pivot_table(
#         index='ward_precinct',
#         columns='candidate_name',
#         values='ballots_candidate_precinct',
#         aggfunc='sum'
#     ).reset_index()

#     df_totals = boe.groupby('ward_precinct').agg({
#         'ballots_district_total':'first',
#         'ballots_candidate_district_total':'first',
#         'ballots_precinct_total':'first'
#     }).reset_index()

#     df = df.merge(df_totals,on="ward_precinct",how="left")


pt_relief_district = pt_relief.groupby(['race_name']).agg({
    'votes':'sum'
    }).reset_index()
pt_relief_choice_total = pt_relief.groupby(['race_name','candidate_name']).agg({
    'votes':'sum'
    }).reset_index()
pt_relief_precinct = pt_relief.groupby(['race_name','ward','precinct']).agg({
    'votes':'sum'
    }).reset_index()

pt_relief = pt_relief.merge(pt_relief_district,on="race_name",how="left",suffixes=("","_district"))
pt_relief = pt_relief.merge(pt_relief_choice_total,on=["race_name","candidate_name"],how="left",suffixes=("","_choice_district_total"))
pt_relief = pt_relief.merge(pt_relief_precinct,on=["race_name","ward","precinct"],how="left",suffixes=("","_precinct"))

rename = {
    'votes':'ballots_choice_precinct',
    'votes_district':'ballots_district_total',
    'votes_choice_district_total':'ballots_choice_district_total',
    'votes_precinct':'ballots_precinct_total_ref'
}

pt_relief = pt_relief.rename(columns=rename)

# pad ward with 1 leading zero and precinct with 2 leading zeros

pt_relief['ward'] = pt_relief['ward'].apply(lambda x: str(x).zfill(2))
pt_relief['precinct'] = pt_relief['precinct'].apply(lambda x: str(x).zfill(3))
pt_relief['ward_precinct'] = pt_relief['ward'] + pt_relief['precinct']

pt_relief_totals = pt_relief.groupby('ward_precinct').agg({
    'ballots_precinct_total_ref':'first',
    'ballots_choice_district_total':'first',
    'ballots_district_total':'first'
    }).reset_index()

pt_relief = pt_relief.pivot_table(
    index='ward_precinct',
    columns='candidate_name',
    values='ballots_choice_precinct',
    aggfunc='sum'
).reset_index()



pt_relief = pt_relief.merge(pt_relief_totals,on="ward_precinct",how="left")

general = general.merge(pt_relief,on="ward_precinct",how="left")

# ersb_10 = gpd.read_file(rf"{mdir}\\ERSB_10_District_Map_FA1_SB_15.shp")
ersb_20 = gpd.read_file(rf"{mdir}\\ERSB_20_Sub_District_Map_FA1_SB_15.shp")
ward_precinct_gdf = gpd.read_file(rf"{mdir}\\ward_precinct.geojson")

# # Set CRS to EPSG:4326
# ersb_10 = ersb_10.to_crs("EPSG:26971")
ersb_20 = ersb_20.to_crs("EPSG:26971")
ward_precinct_gdf = ward_precinct_gdf.to_crs("EPSG:26971")

df['ward_precinct'] = df['ward'].apply(lambda x: str(x).zfill(2)) + df['precinct'].apply(lambda x: str(x).zfill(3))

df_gdf = ward_precinct_gdf.merge(general, on='ward_precinct', how='left')

# Keep ward_precinct,ward_x,precinct_x,geometry,race_name,candidate_name,votes

#df_gdf = df_gdf[['ward_precinct','ward_x','precinct_x','geometry','race_name','candidate_name','votes']]

df_gdf = df_gdf[['ward_precinct','ward_x','precinct_x','geometry','registered_voters','ballots_cast','No',"Yes",'ballots_precinct_total_ref','ballots_choice_district_total','ballots_district_total']]
df_gdf['percent_turnout'] = df_gdf['ballots_cast'] / df_gdf['registered_voters']
df_gdf['percent_yes_precinct'] = df_gdf['Yes'] / df_gdf['ballots_precinct_total_ref']
df_gdf['percent_yes'] = df_gdf['ballots_choice_district_total'] / df_gdf['ballots_district_total']


# # Dates cause a problem with folium. Keep only necessary columns
# ward_precinct_gdf = ward_precinct_gdf[['ward_precinct', 'ward', 'precinct', 'geometry']]
# ward_precinct_gdf['area_precinct'] = ward_precinct_gdf.area

# # Join df to ward_precinct_gdf to get vote totals for each precinct
# ward_precinct_gdf = ward_precinct_gdf.merge(df, on='ward_precinct', how='left')

# # Create a map with ward_precinct and ersb_20 layers
# # Use tiles=None so we can add the satellite layer as the base map
m = folium.Map(location=[41.8781, -87.6298], zoom_start=10, tiles=None)
folium.TileLayer("OpenStreetMap", name="Satellite", control=False).add_to(m)

# # make fill completely transparent. make city boundaries pink and ersb green.
# # add ward and precinct as hovertext to ward_precinct_layer
df_gdf_layer = folium.GeoJson(
    df_gdf,
    name="city",
    tooltip=folium.GeoJsonTooltip(fields=['ward_precinct','registered_voters','ballots_cast','percent_turnout','No','Yes','percent_yes_precinct','percent_yes'], aliases=['Ward-Precinct','Registered Voters','Ballots Cast','Percent Turnout','No to Millionaires Tax','Yes to Millionaires Tax','Percent Yes in Precinct','Percent Yes in District']),
    style_function=lambda x: {'fillColor': 'pink', 'color': 'pink', 'fillOpacity': 0},
).add_to(m)
ersb_layer = folium.GeoJson(
    ersb_20,
    name="ersb_20",
    interactive=False,
    style_function=lambda x: {'fillColor': 'green', 'color': 'green', 'fillOpacity': 0},
).add_to(m)


# # Use the GeoDataFrame for overlay, not the folium layer
# intersection = gpd.overlay(ward_precinct_gdf, ersb_20, how="intersection")

# intersection['area_intersection'] = intersection.area


# intersection["precinct_share"] = intersection['area_intersection'] / intersection['area_precinct']
# intersection["votes_alloc"] = intersection['votes'] * intersection["precinct_share"]

# intersection.explore()

# intersection_layer = folium.GeoJson(
#     intersection,
#     name="intersection",
#     style_function=lambda x: {'fillColor': 'green', 'color': 'green', 'fillOpacity': 0},
# ).add_to(m)

folium.LayerControl().add_to(m)

display(m)

# # Export to geojson

# intersection.to_file(rf"{mdir}\\ward_precinct_ersb20_intersection.geojson", driver="GeoJSON")

# save map as html

#m.save(rf"{mdir}\\ward_precinct_ersb20_intersection.html")

pt_relief
general

KeyError: "['No', 'Yes'] not in index"

In [11]:
# Quick checks: confirm Yes/No columns after pivot and merge
print("pt_relief columns:", list(pt_relief.columns))
print("general columns:", list(general.columns))
print("df_gdf columns:", list(df_gdf.columns))

# Peek at candidate_name values that become columns in the pivot
print("candidate_name unique:", sorted(pt_relief['candidate_name'].dropna().unique())[:20])
print("... total unique:", pt_relief['candidate_name'].nunique())

pt_relief columns: ['ward_precinct', 'No                                                                         ', 'Yes                                                                        ', 'ballots_precinct_total_ref', 'ballots_choice_district_total', 'ballots_district_total']
general columns: ['ward', 'precinct', 'registered_voters', 'ballots_cast', 'ward_precinct', 'No                                                                         ', 'Yes                                                                        ', 'ballots_precinct_total_ref', 'ballots_choice_district_total', 'ballots_district_total']
df_gdf columns: [':id', ':version', ':created_at', ':updated_at', 'shape_leng', 'shape_area', 'ward_precinct', 'ward_x', 'precinct_x', 'geometry', 'ward_y', 'precinct_y', 'registered_voters', 'ballots_cast', 'No                                                                         ', 'Yes                                                                        ', 'ballots_pr

KeyError: 'candidate_name'